# 1. Run General Linear Model:

Create the video beta maps, which will be submitted to the LASSOPCR_Searchlight analysis in step 2

*Yiyu Wang 2021 Nov*



In [2]:
import os
import glob
import nibabel as nib
import numpy as np
import pandas as pd
import copy

import nilearn
from nilearn.image import smooth_img, resample_to_img
from nilearn import plotting
from nilearn.plotting import plot_glass_brain
from nilearn.plotting import plot_stat_map
from nilearn.plotting import plot_design_matrix
from nilearn.masking import apply_mask
from nilearn.input_data import NiftiMasker
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from scipy.stats import norm


import gzip

import matplotlib.pyplot as plt
from os.path import join
import warnings
warnings.filterwarnings('ignore')

In [2]:
# directories

data_dir ='fmriprep/'
logfiles_dir = 'logfiles/'
confounds_dir = 'confounds/'

visual_reg_dir = 'visual_regressors/'

# masks:
mask_path = 'masks/FSL_binary_MNI152_T1_3mm_brain.nii.gz'
mask = nib.load(mask_path)
# resample a gray matter mask
from nilearn import datasets
gm_mask = datasets.fetch_icbm152_brain_gm_mask()
gm_mask_img = resample_to_img(
    gm_mask, mask, interpolation='nearest')


# subjects information:
subjects_list = ['04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','23','25','26','28','29']
sample_n = len(subjects_list)
print("subjects in this analysis:")
print(subjects_list)
print(f"**** n = {sample_n} *****" )

# fmri data info:
TR = .001
N_TR = 552
TR_Length = 1
TR_IN_MS = int(TR_Length/TR)

fwhm = 6

confounds_of_interest = ['CSF',
                        'WhiteMatter',
                        'X', 
                        'Y', 
                        'Z',
                        'RotX',
                        'RotY',
                        'RotZ','FramewiseDisplacement']


subjects in this analysis:
['04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '23', '25', '26', '28', '29']
**** n = 21 *****


In [3]:
# functions to grab regressors
def AddSteadyStateOutliers(columns_of_interest, all_columns):
    new_columns = copy.deepcopy(columns_of_interest)
    for column in all_columns:
        if 'Outlier' in column:
            new_columns.append(column)
            
    return new_columns


def AddVisualProperty(cov, s, run):
    new_cov = copy.deepcopy(cov)
    visual_reg = pd.read_csv(visual_reg_dir + f'subject_{s}_run_{run}.csv')
    new_cov = pd.concat([cov, visual_reg], axis =1)
    return new_cov

def CreateConfoundMatrix(confound_file_path, 
                         confounds_of_interest, s, run, add_vis=False):
    
    
    confounds = pd.read_csv(confound_file_path, sep='\t')
    
    confounds_of_interest = AddSteadyStateOutliers(confounds_of_interest, confounds.columns)
    
    cov = confounds[confounds_of_interest]
    if add_vis:
        cov = AddVisualProperty(cov, s, run)
    cov.values[np.isnan(cov.values)]=0
    return cov


In [10]:
# functions preprocess logfiles to events:

logfile_headers =np.array(['video_name', 'run_number', 'video_category', 'experimenter_high_low', 'same_category_prior', 
                  'cue_onset','cue_offset', 'expected_fear', 'expected_fear_RT',
                 'video_onset', 'video_offset', 'fear_rating', 'arousal_rating', 'valence_rating',
                  'fear_rating_RT', 'arousal_rating_RT', 'valence_rating_RT'])

def get_video_n(video_name):
    # category
    if video_name[0:2]=='he':
        v_num = 0
    elif video_name[0:2] == 'so':
        v_num = 12
    else:
        v_num = 24
    num = video_name[-5:-4]
    
    # high vs low
    if video_name[-9:-7] == 'lo':
        v_num = v_num + 6
        
    # special case: social_high_4_replacement = 16
    if num == 't':
        v_num = 16
    else:
        num = int(num)
        v_num = v_num + num
    return int(v_num)

def get_vcat(cn):
    if cn == 1:
        cat = 'Heights'

    elif cn == 2:
        cat = 'Social'

    elif cn == 3:
        cat = 'Spiders'
    else:
        print('no such category number!')
    return cat 

def parse_task_lines(lines, headers):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        video_name = cols[int(np.where(headers == 'video_name')[0])]
        video_number = get_video_n(video_name)
        
        category_number = int(cols[int(np.where(headers == 'video_category')[0])])
        video_category = get_vcat(category_number)
        
        video_onset = float(cols[int(np.where(headers == 'video_onset')[0])])
        video_offset = float(cols[int(np.where(headers == 'video_offset')[0])])  
        video_duration = video_offset - video_onset

        run = int(cols[int(np.where(headers == 'run_number')[0])])
        fear_rating = abs(float(cols[int(np.where(headers == 'fear_rating')[0])]))
        arousal_rating = abs(float(cols[int(np.where(headers == 'arousal_rating')[0])]))
        valence_rating = abs(float(cols[int(np.where(headers == 'valence_rating')[0])]))

        yield [video_onset, video_duration, video_number, run]
            

def create_events_dataframe(task_csv, run, model):   
    task_lines =[]       
    # df = pd.DataFrame(columns=['onset','duration','trial_type'])
    with open(task_csv, 'r') as task_csv_file:
        task_lines.append(list(parse_task_lines(task_csv_file.readlines()[1:], logfile_headers, model)))

    df = pd.DataFrame(task_lines[0], columns=['onset','duration','trial_type','run'])
    df= df[df['run']==run].drop(columns=['run'])
    return df


## Run GLM:

In [5]:
verbose = False
dm_name = 'OneRegPerVid_VisReg'
res_dir = f'results/{dm_name}/1stLvl/'
if not os.path.isdir(res_dir):
        os.mkdir(f'results/{dm_name}/')
        os.mkdir(res_dir)


for s in subjects_list:
    print(f'running subject {s}')
    sub_output_dir = res_dir + f'/{s}/'
    if not os.path.isdir(sub_output_dir):
        os.mkdir(sub_output_dir)
        
    task_file = glob.glob(logfiles_dir + f'AffVids_logfile_{s}_edited.txt') 
    task_csv = task_file[0]
    
    for run in [1,2,3]:
        events = create_events_dataframe(task_csv, run)
        ##get confounds info:
        confounds_str = f'sub-{s}_task-AffVids_run-0{run}_bold_confounds.tsv'
        cov = CreateConfoundMatrix(confounds_dir + confounds_str, confounds_of_interest, s, run, add_vis=True)

        fmri_glm = FirstLevelModel(t_r=1,
                           noise_model='ar3',
                           standardize=True,
                           hrf_model='spm',
                           drift_model='cosine',
                           high_pass=.01, mask_img=gm_mask_img,smoothing_fwhm=fwhm)
        
        ###############################################
        # Change this to match your run nifti files:
        func_str = f'{s}/run{run}.nii.gz'
        func_path = data_dir + func_str
        ###############################################
        fmri_img = nib.load(func_path)
        fmri_glm = fmri_glm.fit(fmri_img, events, confounds=cov)
        
        # save design_matrix for every run
        design_matrix = fmri_glm.design_matrices_[0]

        plot_design_matrix(design_matrix, output_file=join(sub_output_dir, f'design_matrix_run{run}.png'))
        contrast_matrix = np.eye(design_matrix.shape[1])
        # extract the betas
        for i in range(12):
            if verbose:
                print(f'saving beta for video {design_matrix.columns[i]}')
            eff = fmri_glm.compute_contrast(contrast_matrix[i],output_type='stat')
            nii_file_path = sub_output_dir + f'/sub-{s}_run-{run}_beta_video-{design_matrix.columns[i]}_gm_visreg.nii.gz'
            nib.save(eff, nii_file_path)

        
        

running subject 04
saving beta for video 1
saving beta for video 6
saving beta for video 7
saving beta for video 12
saving beta for video 16
saving beta for video 17
saving beta for video 19
saving beta for video 22
saving beta for video 25
saving beta for video 30
saving beta for video 31
saving beta for video 34
saving beta for video 3
saving beta for video 5
saving beta for video 8
saving beta for video 11
saving beta for video 14
saving beta for video 15
saving beta for video 21
saving beta for video 24
saving beta for video 26
saving beta for video 27
saving beta for video 32
saving beta for video 35
saving beta for video 2
saving beta for video 4
saving beta for video 9
saving beta for video 10
saving beta for video 13
saving beta for video 18
saving beta for video 20
saving beta for video 23
saving beta for video 28
saving beta for video 29
saving beta for video 33
saving beta for video 36
running subject 05
saving beta for video 1
saving beta for video 5
saving beta for video 7